In [2]:
from datasets import load_dataset
db=load_dataset('squad')
db

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [3]:
db['train'][0]['context']

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [4]:
db['train'][0]['question']

'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'

In [7]:
db['train'][0]['answers']['text']

['Saint Bernadette Soubirous']

In [44]:
train_db=db['train']
val_db=db['validation']

In [17]:
train_db

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [11]:
import torch
device=['cuda' if torch.cuda.is_available() else "cpu"]

In [45]:
from transformers import AutoTokenizer
model_name = "iarfmoose/t5-base-question-generator" 

tokenizer=AutoTokenizer.from_pretrained(model_name)

def tokenize_fun(data,max_length=128):
    return tokenizer(
        data,
        padding='max_length',
        max_length=128,
        truncation=True,
        return_tensors='pt',
    )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [48]:
def organizeQuestion(data):
    prefix='Generate question for this context'
    inputs=[prefix+ doc for doc in data['context']]
    model_inputs=tokenize_fun(inputs,max_length=512)
    labels=tokenize_fun(data['question'])
    model_inputs['labels']=labels['input_ids']
    return model_inputs

In [49]:
train_db=train_db.map(organizeQuestion,batched=True,batch_size=32,remove_columns=train_db.column_names)
val_db=val_db.map(organizeQuestion,batched=True,batch_size=32,remove_columns=val_db.column_names)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [31]:
train_db

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 2738
})

In [37]:
val_db

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 331
})

In [51]:
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer, DataCollatorWithPadding

model=AutoModelForSeq2SeqLM.from_pretrained(model_name)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [41]:
train_db

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 2738
})

In [52]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='./My_T5QG',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=10,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    learning_rate=2e-3
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_db,         # training dataset
    eval_dataset=val_db,             # evaluation dataset
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/nwe/newgalaxie/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


/home/nwe/newgalaxie/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/nwe/newgalaxie/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/nwe/newgalaxie/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/nwe/newgalaxie/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input 

KeyboardInterrupt: 

In [53]:
trainer.evaluate()

{'eval_loss': 0.2682490050792694}

In [54]:
last_checkpoint = "./My_T5QG/checkpoint-10000"

finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(last_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(last_checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [56]:
val_db

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10570
})

In [57]:
prefix='Generate question for this context'
context=db['validation'][10]['context']
inputs=prefix+context
inputs = tokenizer(inputs, return_tensors="pt")
outputs = finetuned_model.generate(**inputs)
answer = tokenizer.decode(outputs[0])
from textwrap import fill
print(fill(answer, width=80))

/home/nwe/newgalaxie/lib/python3.8/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> What was the name of the American football team that won the 2015
season?</s>


In [ ]:
#to run again with compute metric
#https://www.datacamp.com/tutorial/flan-t5-tutorial